#Detectron2 Installation <br>
1) Select the GPU as hardware in the runtime<br>
2) Run the following cell.<br>
NB: The runtime will be restarted at the end of this installation which requires few minutes.


In [ ]:
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
exit(0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 3.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-6z54jtrt
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-6z54jtrt
  Resolved https://github.com/facebookresearch/detectron2.git to commit 8c4a333ceb8df05348759443d0206302485890e0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.

# Check the installation
The right output running the following cell should be:<br><br>
nvcc: NVIDIA (R) Cuda compiler driver<br>
Copyright (c) 2005-2020 NVIDIA Corporation<br>
Built on Wed_Sep_21_10:33:58_PDT_2022<br>
Cuda compilation tools, release 11.8, V11.8.89<br>
Build cuda_11.8.r11.8/compiler.31833905_0<br>
torch:  2.0 ; cuda:  cu118<br>
detectron2: 0.6<br>

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


#Environment Setup

Replace at the following path ../usr/local/lib/python3.10/dist-packages/detectron2/modeling/meta_arch/ the **rcnn.py** script with my **rcnn.py**.<br>
Do the same for:<br> 
**roi_heads.py** file at the path ../usr/local/lib/python3.10/dist-packages/detectron2/modeling/roi_heads/ <br>
**rpn.py** file at the path ../usr/local/lib/python3.10/dist-packages/detectron2/modeling/proposal_generator/ <br>

Inside the folder ../usr/local/lib/python3.10/dist-packages/detectron2/modeling/ create a folder called **da_modules** and upload the four files:<br>
grad_rev_layer.py<br>
image_level_discriminators.py<br>
instance_level_discriminators.py<br>
consistency_regularization_loss.py<br>
Restart the runtime<br>
Load the dataset in Google Drive and import it running the cell below.

NB: python version can change

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import cv2
import random
from detectron2 import model_zoo
from detectron2.config import get_cfg
import logging
import os
from collections import OrderedDict
from torch.nn.parallel import DistributedDataParallel
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.data import MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.utils.events import EventStorage
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch
import torch, torchvision
from detectron2.data.datasets import register_coco_instances, load_coco_json, register_pascal_voc
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Register your Dataset
Run the following cell according to your dataset path. <br>
If your annotations are in PASCAL VOC use **register_pascal_voc** otherwise **register_coco_instances** if they are in COCO format.



In [ ]:
register_pascal_voc("city_trainS", "drive/My Drive/cityscape/", "train_s", 2007, ['car','person','rider','truck','bus','train','motorcycle','bicycle'])
register_pascal_voc("city_trainT", "drive/My Drive/cityscape/", "train_t", 2007, ['car','person','rider','truck','bus','train','motorcycle','bicycle'])

register_pascal_voc("city_testT", "drive/My Drive/cityscape/", "test_t", 2007, ['car','person','rider','truck','bus','train','motorcycle','bicycle'])

In [ ]:
#This block is an example of how you should use this API with COCO annotations
register_coco_instances("dataset_train_synthetic", {}, "drive/My Drive/Bellomo_Dataset_UDA/synthetic/Object_annotations/Training_annotations.json", "./drive/My Drive/Bellomo_Dataset_UDA/synthetic/images")
register_coco_instances("dataset_train_real", {}, "drive/My Drive/Bellomo_Dataset_UDA/real_hololens/training/training_set.json", "./drive/My Drive/Bellomo_Dataset_UDA/real_hololens/training")

register_coco_instances("dataset_test_real", {}, "drive/My Drive/Bellomo_Dataset_UDA/real_hololens/test/test_set.json", "./drive/My Drive/Bellomo_Dataset_UDA/real_hololens/test")

#Training Loop Definition
Run the following block

In [ ]:
logger = logging.getLogger("detectron2")

def do_train(cfg_source, cfg_target, model, resume = False):

    model.train()
    optimizer = build_optimizer(cfg_source, model)
    scheduler = build_lr_scheduler(cfg_source, optimizer)
    checkpointer = DetectionCheckpointer(model, cfg_source.OUTPUT_DIR, optimizer=optimizer, scheduler=scheduler)

    start_iter = (checkpointer.resume_or_load(cfg_source.MODEL.WEIGHTS, resume=resume).get("iteration", -1) + 1)
    max_iter = cfg_source.SOLVER.MAX_ITER

    periodic_checkpointer = PeriodicCheckpointer(checkpointer, cfg_source.SOLVER.CHECKPOINT_PERIOD, max_iter=max_iter)
    writers = default_writers(cfg_source.OUTPUT_DIR, max_iter) if comm.is_main_process() else []

    data_loader_source = build_detection_train_loader(cfg_source)
    data_loader_target = build_detection_train_loader(cfg_target)
    logger.info("Starting training from iteration {}".format(start_iter))

    lambda_hyper = 0.1

    with EventStorage(start_iter) as storage:
        for data_source, data_target, iteration in zip(data_loader_source, data_loader_target, range(start_iter, max_iter)):
            storage.iter = iteration

            loss_dict = model(data_source, False, 1)
            loss_dict_target = model(data_target, True, 1)
            
            loss_dict["loss_image_d"] += loss_dict_target["loss_image_d"]
            loss_dict["loss_instance_d"] += loss_dict_target["loss_instance_d"]
            loss_dict["loss_consistency_d"] += loss_dict_target["loss_consistency_d"]

            loss_dict["loss_image_d"] *= ( 0.5 * lambda_hyper)
            loss_dict["loss_instance_d"] *= ( 0.5 * lambda_hyper)
            loss_dict["loss_consistency_d"] *= ( 0.5 * lambda_hyper)

            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {k: v.item() for k, v in comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                storage.put_scalars(total_loss=losses_reduced, **loss_dict_reduced)

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            storage.put_scalar("lr", optimizer.param_groups[0]["lr"], smoothing_hint=False)
            scheduler.step()

            if iteration - start_iter > 5 and ((iteration + 1) % 50 == 0 or iteration == max_iter - 1):
                for writer in writers:
                    writer.write()
            periodic_checkpointer.step(iteration)

#Configuration Definition
Define the configuration for the source (cfg_source) and target dataset (cfg_target). The cfg_source contains also the parameters which will be used by the network such us:<br>
learning rate, number of training iterations, weight decay, number of classes etc...

## Backbone for Faster RCNN
this implementations works with three kind of backbone:<br> FPN: "COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"<br>
DC5: "COCO-Detection/faster_rcnn_R_50_DC5_1x.yaml"<br>
C4: "COCO-Detection/faster_rcnn_R_50_C4_1x.yaml"<br>

You can also use their variants such us faster_rcnn_R_101_C4_3x, faster_rcnn_R_50_DC5_3x, faster_rcnn_R_101_DC5_3x, etc...





In [ ]:
cfg_source = get_cfg()
cfg_source.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg_source.DATASETS.TRAIN = ("city_trainS",)
cfg_source.DATALOADER.NUM_WORKERS = 2
cfg_source.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
cfg_source.SOLVER.IMS_PER_BATCH = 4
cfg_source.SOLVER.BASE_LR = 0.0005
cfg_source.SOLVER.WARMUP_FACTOR = 1.0 / 100
cfg_source.SOLVER.WARMUP_ITERS = 1000
cfg_source.SOLVER.MAX_ITER = 5000
cfg_source.INPUT.MIN_SIZE_TRAIN = (600,)
cfg_source.INPUT.MIN_SIZE_TEST = 0
os.makedirs(cfg_source.OUTPUT_DIR, exist_ok=True)
cfg_source.MODEL.ROI_HEADS.NUM_CLASSES = 8
model = build_model(cfg_source)

cfg_target = get_cfg()
cfg_target.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg_target.DATASETS.TRAIN = ("city_trainT",)
cfg_target.INPUT.MIN_SIZE_TRAIN = (600,)
cfg_target.DATALOADER.NUM_WORKERS = 0
cfg_target.SOLVER.IMS_PER_BATCH = 4

build_resnet_fpn_backbone


Large datasets can require much time to be loaded from google drive. Please be patient and restart the runtime if the training does not start in few minutes.

In [ ]:
do_train(cfg_source,cfg_target,model)

WARNING [09/17 21:21:54 d2.solver.build]: SOLVER.STEPS contains values larger than SOLVER.MAX_ITER. These values will be ignored.
[09/17 21:21:54 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_50_FPN_1x/137257794/model_final_b275ba.pkl ...


discriminator.reducer.0.weight
discriminator.reducer.2.weight
discriminator.reducer.5.weight
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.discriminatorProposal.reducer.0.weight
roi_heads.discriminatorProposal.reducer.2.weight


[09/17 21:22:02 d2.data.build]: Removed 10 images with no usable annotations. 2965 images left.
[09/17 21:22:03 d2.data.build]: Distribution of instances among all 8 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    car     | 27155        |   person   | 17994        |   rider    | 1807         |
|   truck    | 489          |    bus     | 385          |   train    | 171          |
| motorcycle | 739          |  bicycle   | 3729         |            |              |
|   total    | 52469        |            |              |            |              |
[09/17 21:22:03 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(600,), max_size=1333, sample_style='choice'), RandomFlip()]
[09/17 21:22:03 d2.data.build]: Using training sampler TrainingSampler
[09/17 21:22:03 d2.data.common]: Serializing th

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[09/17 21:24:07 d2.utils.events]:  iter: 49  total_loss: 1.932  loss_cls: 0.7193  loss_box_reg: 0.7616  loss_rpn_cls: 0.0591  loss_rpn_loc: 0.19  loss_image_d: 0.0866  loss_instance_d: 0.08039     lr: 0.0004902  max_mem: 5695M
[09/17 21:26:01 d2.utils.events]:  eta: 1:49:29  iter: 99  total_loss: 1.557  loss_cls: 0.4791  loss_box_reg: 0.7046  loss_rpn_cls: 0.05925  loss_rpn_loc: 0.1825  loss_image_d: 0.08659  loss_instance_d: 0.02086     lr: 0.0005  max_mem: 5695M
[09/17 21:27:50 d2.utils.events]:  eta: 1:43:25  iter: 149  total_loss: 1.246  loss_cls: 0.3636  loss_box_reg: 0.5399  loss_rpn_cls: 0.05065  loss_rpn_loc: 0.1825  loss_image_d: 0.08658  loss_instance_d: 0.00762     lr: 0.0005  max_mem: 5695M
[09/17 21:29:41 d2.utils.events]:  eta: 1:43:41  iter: 199  total_loss: 1.282  loss_cls: 0.366  loss_box_reg: 0.4881  loss_rpn_cls: 0.05973  loss_rpn_loc: 0.181  loss_image_d: 0.08657  loss_instance_d: 0.005387     lr: 0.0005  max_mem: 5695M
[09/17 21:31:34 d2.utils.events]:  eta: 1:43:2

##Evalutate the performance
runt the PascalVOCDetectionEvaluator if your annotations are in PASCAL VOC otherwhise run the COCOEvaluator<br>

The mAP50 is the object detection result on the dataset. In this case, for the cityscape dataset, the result is 39.5%


In [ ]:
#PASCAL VOC evaluation
from detectron2.evaluation import inference_on_dataset, PascalVOCDetectionEvaluator
from detectron2.data import build_detection_test_loader
evaluator = PascalVOCDetectionEvaluator("city_testT")
val_loader = build_detection_test_loader(cfg_source, "city_testT")
res = inference_on_dataset(model, val_loader, evaluator)
print(res)

[09/17 23:00:48 d2.data.build]: Distribution of instances among all 8 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    car     | 4667         |   person   | 3419         |   rider    | 556          |
|   truck    | 93           |    bus     | 98           |   train    | 23           |
| motorcycle | 149          |  bicycle   | 1175         |            |              |
|   total    | 10180        |            |              |            |              |
[09/17 23:00:48 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(0, 0), max_size=1333, sample_style='choice')]
[09/17 23:00:48 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/17 23:00:48 d2.data.common]: Serializing 500 elements to byte tensors and concatenating them all ...
[09/

In [ ]:
#COCO evaluation example
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
evaluator = COCOEvaluator("dataset_test_real", cfg_source, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg_source, "dataset_test_real")
inference_on_dataset(model, val_loader, evaluator)